```
ME72: Maestría en Métodos Cuantitativos para la Gestión y Análisis de Datos
M72109: Analisis de Datos no Estructurados
Universidad de Buenos Aires - Facultad de Ciencias Economicas (UBA-FCE)
Año: 2020
Profesor: Facundo Santiago, Javier Ignacio Garcia Fronti
```

# Word2Vec - Modelos basados en secuencias

## Prepración del ambiente

In [1]:
import warnings
warnings.filterwarnings('ignore')

Descarguemos algunos fragmentos de código para simplificar el trabajo

In [53]:
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Utils/PadSequenceTransformer.py --directory-prefix ./Utils/
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Utils/Word2VecVectorizer.py --directory-prefix ./Utils/

--2020-09-04 18:30:34--  https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Utils/PadSequenceTransformer.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.208.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.208.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 980 [text/plain]
Saving to: ‘./Utils/PadSequenceTransformer.py.1’

PadSequenceTransfor 100%[===================>]     980  --.-KB/s    in 0s      

2020-09-04 18:30:35 (64.0 MB/s) - ‘./Utils/PadSequenceTransformer.py.1’ saved [980/980]

--2020-09-04 18:30:35--  https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Utils/Word2VecVectorizer.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.208.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.208.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3612 (3.5K) [text/plain]
Saving to: ‘./Utils/Wo

Descargamos el set de datos

In [ ]:
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Datasets/speech-transcripts-es/transcripts.csv --directory-prefix ./Datasets/mascorpus/

Descargamos nuestros vectores de word2vec en español

In [ ]:
!mkdir -p ./Models/Word2Vec
!wget -N https://santiagxf.blob.core.windows.net/public/Word2Vec/model.bin --directory-prefix ./Models/Word2Vec

## Exploración del set de datos

Cargamos el set de datos

In [2]:
import pandas as pd

transcripts = pd.read_csv('Datasets/speech-transcripts-es/transcripts.csv')

Veamos como luce este nuevo set de datos

In [3]:
transcripts.head(5)

,transcript
0,y aquí en dos palotadas hemos encontrado robus...
1,cuando los consejeros escucharon aquello queda...
2,su mujer con la cara entre las manos
3,y otros que se podían echar a la oreja de un toro
4,al oír mis pasos alzó la cabeza


## Creación de un modelo de lenguaje usando Word2Vec

En esta oportunidad utilizaremos Word2Vec utilizando un modelo pre-entrenado para el idioma español. Adicionalmente, vemos que este vectorizer tiene el parametro sequence_to_idx en Verdadero. Esto significa que no queremos que como salida obtengamos los vectores de Word2Vec, sino que queremos "el indice" que se corresponde a la palabra en una matriz de indice-palabra/vectores.

In [4]:
from Utils.Word2VecVectorizer import Word2VecVectorizer

w2v = Word2VecVectorizer(model_path='./Models/Word2Vec/model.bin', sequence_to_idx=True)
pretrained_weights = w2v.get_weights()
vocab_size, emdedding_size = pretrained_weights.shape

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.word_index = { key: word.index for key,word in w2v.embeddings.vocab.items() }

In [6]:
tensors = tokenizer.texts_to_sequences(transcripts['transcript'].to_numpy())

In [7]:
max_sequence = max([len(s) for s in tensors])

In [10]:
import tensorflow.keras as keras
from tensorflow.keras.layers import Embedding, LSTM, Dense, Activation, SpatialDropout1D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.callbacks import LambdaCallback

In [11]:
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=emdedding_size, weights=[pretrained_weights]),
    SpatialDropout1D(0.2),
    LSTM(units=emdedding_size, return_sequences=False),
    Dense(units=vocab_size, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

In [12]:
def sample(preds, temperature=1.0):
    if temperature <= 0:
        return np.argmax(preds)
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate_next(text, num_generated=3):
    word_idxs = tokenizer.texts_to_sequences([text])
    for i in range(num_generated):
        inputs = np.array(pad_sequences(word_idxs, maxlen=40, padding="post"))
        prediction = model.predict(inputs)
        idx = sample(prediction[-1], temperature=0.7)
        word_idxs = np.concatenate((word_idxs, idx.reshape(-1,1)), axis=1)
    
    return [' '.join([w2v.idx2word(idx) for idx in seq]) for seq in word_idxs]

def on_epoch_end(epoch, _):
    print('\nGenerating text after epoch: %d' % epoch)
    texts = [
        'con la cara',
        'Nunca estuvimos tan',
        'ojo por ojo y diente por',
      ]
    for text in texts:
        sample = generate_next(text)
        print('%s... -> %s' % (text, sample))

In [13]:
import numpy as np

train_x = np.zeros([len(tensors), max_sequence], dtype=np.int32)
train_y = np.zeros([len(tensors)], dtype=np.int32)
for i, sentence in enumerate(tensors):
    for t, word in enumerate(sentence[:-1]):
        train_x[i, t] = word
    train_y[i] = sentence[-1]

In [14]:
model.fit(train_x, train_y,
          batch_size=128,
          epochs=20,
          callbacks=[LambdaCallback(on_epoch_end=on_epoch_end)])

Epoch 1/20
882/882 [==============================] - ETA: 0s - loss: 10.4651
Generating text after epoch: 0
con la cara... -> ['con la cara persona fin puesto']
Nunca estuvimos tan... -> ['nunca estuvimos tan mundo de mano']
ojo por ojo y diente por... -> ['ojo por ojo y diente por equitación padres rey']
882/882 [==============================] - 1102s 1s/step - loss: 10.4651
Epoch 2/20
882/882 [==============================] - ETA: 0s - loss: 9.0888
Generating text after epoch: 1
con la cara... -> ['con la cara corazón presente madrí']
Nunca estuvimos tan... -> ['nunca estuvimos tan cogieran tío niños']
ojo por ojo y diente por... -> ['ojo por ojo y diente por vida ello esperanza']
882/882 [==============================] - 1101s 1s/step - loss: 9.0888
Epoch 3/20
882/882 [==============================] - ETA: 0s - loss: 8.9803
Generating text after epoch: 2
con la cara... -> ['con la cara camino cuatro lunetas']
Nunca estuvimos tan... -> ['nunca estuvimos tan causas familia oídos'

KeyboardInterrupt: 